In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import timeit

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, precision_score, recall_score, accuracy_score, auc
from hmeasure import h_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, precision_recall_fscore_support,roc_auc_score, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras.wrappers.scikit_learn import KerasClassifier
from keras import initializers
from keras.models import Sequential
from keras.optimizers import Adam

# Read the data using csv
path_train = r'C:\Users\...\credit_risk_dataset_cleanedtraining.csv'
df_train = pd.read_csv(path_train, encoding = "ISO-8859-1")

path_test = r'C:\Users\...\credit_risk_dataset_cleanedtesting.csv'
df_test = pd.read_csv(path_test, encoding = "ISO-8859-1")

# Prin 5 forst lines of df
df_train.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,person_home_ownership,loan_intent,loan_grade,cb_person_default_on_file,loan_status
0,23,120000,1.0,25600,12.69,0.21,3,3,1,1,0,0
1,21,42500,3.0,25000,12.73,0.59,3,3,1,2,1,1
2,26,48000,2.0,25000,9.64,0.52,4,3,1,1,0,1
3,25,42360,1.0,25000,16.35,0.59,3,3,1,4,1,1
4,23,50000,1.0,25000,7.90,0.50,3,3,0,0,0,1


In [2]:
X_test, y_test = df_test.iloc[:,:-1], df_test.iloc[:,-1]

X_train, y_train = df_train.iloc[:,:-1], df_train.iloc[:,-1]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2)

In [3]:
class tuning_methods:
    
    def __init__(self, X_train, X_val, y_train,  y_val, X_test, y_test):
        
        self.X_train = X_train
        self.X_val = X_val
        self.X_test = X_test
        self.y_test = y_test
        self.y_train = y_train
        self.y_val = y_val
    
  # Define the Keras model
    def create_model(self, units_1, alpha_1, dropout_1, l2_kern_1, l2_bias_1,
                 units_2, alpha_2, dropout_2, l2_kern_2, l2_bias_2,
                 units_3, alpha_3, dropout_3, l2_kern_3, l2_bias_3,
                 lr, batch_size):        # 
    
        model = Sequential()
    
        model.add(Dense(units_1, 
                input_shape=(X_train.shape[1],), 
                activation = LeakyReLU(alpha=alpha_1),
                kernel_regularizer = l2(l2_kern_1),
                bias_regularizer = l2(l2_bias_1),
                kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05)
        ))
        model.add(Dropout(dropout_1))
    
        model.add(Dense(units_2,
                    activation =LeakyReLU(alpha=alpha_2),
                    kernel_regularizer = l2(l2_kern_2),
                    bias_regularizer = l2(l2_bias_2),
                    kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05)
        ))
        model.add(Dropout(dropout_2))
    
        model.add(Dense(units_3,
                        activation =LeakyReLU(alpha=alpha_3),
                        kernel_regularizer = l2(l2_kern_3),
                        bias_regularizer = l2(l2_bias_3),
                        kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05)
        ))
        model.add(Dropout(dropout_3))
    
        model.add(Dense(1, activation='sigmoid'))
    
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', 
                  metrics=['AUC'])
        return model

    
    # Define the Optuna study
    def objective(self, trial):
        
        # Define the pipeline
        pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))
                ])
        # Define hyperparameters range values for tuning
        units_1 = trial.suggest_int('units_1', 2, 40)
        units_2 = trial.suggest_int('units_2', 2, 40)
        units_3 = trial.suggest_int('units_3', 2, 40)
        alpha_1 = trial.suggest_float('alpha_1', 0.01, 0.1)
        alpha_2 = trial.suggest_float('alpha_2', 0.01, 0.1)
        alpha_3 = trial.suggest_float('alpha_3', 0.01, 0.1)
        dropout_1 = trial.suggest_float("dropout_1", 0.2, 0.5)    
        dropout_2 = trial.suggest_float("dropout_2", 0.2, 0.5)    
        dropout_3 = trial.suggest_float("dropout_3", 0.2, 0.5)    
        l2_kern_1 = trial.suggest_float("l2_kern_1",  1e-5, 1e-2, log=True)
        l2_kern_2 = trial.suggest_float("l2_kern_2",  1e-5, 1e-2, log=True)
        l2_kern_3 = trial.suggest_float("l2_kern_3",  1e-5, 1e-2, log=True)
        l2_bias_1 = trial.suggest_float("l2_bias_1",  1e-5, 1e-2, log=True)
        l2_bias_2 = trial.suggest_float("l2_bias_2",  1e-5, 1e-2, log=True)
        l2_bias_3 = trial.suggest_float("l2_bias_3",  1e-5, 1e-2, log=True)
        lr = trial.suggest_float("lr", 5e-5, 1e-2, log=True)
        batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024])
        epochs = 200
        validation_data = (self.X_val, self.y_val)
        early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        
        # Set parameters into pipeline
        pipeline.set_params(classifier__units_1=units_1,
                        classifier__alpha_1=alpha_1,
                        classifier__l2_kern_1=l2_kern_1,
                        classifier__l2_bias_1=l2_bias_1,
                        classifier__dropout_1=dropout_1,
                        classifier__units_2=units_2,
                        classifier__alpha_2=alpha_2,
                        classifier__l2_kern_2=l2_kern_2,
                        classifier__l2_bias_2=l2_bias_2,
                        classifier__dropout_2=dropout_2,
                        classifier__units_3=units_3,
                        classifier__alpha_3=alpha_3,
                        classifier__l2_kern_3=l2_kern_3,
                        classifier__l2_bias_3=l2_bias_3,
                        classifier__dropout_3=dropout_3,
                        classifier__lr=lr,
                        classifier__epochs = epochs,
                        classifier__batch_size = batch_size,
                        classifier__validation_data = validation_data,
                        classifier__callbacks=[early_stopping])
        
        # Apply cross validation
        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2023)
        scores = cross_val_score(pipeline, self.X_train, self.y_train, cv=kfold, scoring="roc_auc", error_score='raise')
    
        if trial.should_prune():
                raise optuna.TrialPruned()

        return np.mean(scores)

    def Tuning(self, n_trials=100, sampler = optuna.samplers.TPESampler()):
        
        start = timeit.default_timer()

        study = optuna.create_study(direction="maximize",             
                            sampler=sampler, 
                            pruner= optuna.pruners.MedianPruner()
                           )
        study.optimize(self.objective, n_trials=n_trials)  # to be converged it needs atleast 200 trials 

        print("Number of finished trials: {}".format(len(study.trials)))

        print("Best trial:")
        trial = study.best_trial

        print("  Value: {}".format(trial.value))

        stop = timeit.default_timer()

        time_opt = stop - start
        params_TPE = []

        for key, value in trial.params.items():
            params_TPE.append([key,value])
            print("    {}: {}".format(key, value))
            
        params_TPE.append(["time",round(time_opt/60,2)])

        return params_TPE
    
    def Best_MLP_model(self, params):
        
        # Define hyperpameters
        units_1 = params[0][1]
        units_2 = params[1][1]
        units_3 = params[2][1]
        alpha_1 = np.round(params[3][1],3)
        alpha_2 = np.round(params[4][1],3)
        alpha_3 = np.round(params[5][1],3)
        dropout_1 = np.round(params[6][1],2)
        dropout_2 = np.round(params[7][1],2)
        dropout_3 = np.round(params[8][1],2)
        kernel_regularizer_1=np.round(params[9][1],5)
        kernel_regularizer_2=np.round(params[10][1],5)
        kernel_regularizer_3=np.round(params[11][1],5)
        bias_regularizer_1=np.round(params[12][1],5)
        bias_regularizer_2=np.round(params[13][1],5)
        bias_regularizer_3=np.round(params[14][1],5)
        lr = np.round(params[15][1],5)
        batchs = params[16][1]
    
        # Define the pipeline
        pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))
                ])

        # Set hyperparameters
        epochs = 200
        validation_data = (self.X_val, self.y_val)
        early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        pipeline.set_params(classifier__units_1=units_1,
                            classifier__alpha_1 = alpha_1,
                            classifier__l2_kern_1=kernel_regularizer_1,
                            classifier__l2_bias_1=bias_regularizer_1,
                            classifier__dropout_1=dropout_1,
                            classifier__units_2=units_2,
                            classifier__alpha_2=alpha_2,
                            classifier__l2_kern_2=kernel_regularizer_2,
                            classifier__l2_bias_2=bias_regularizer_2,
                            classifier__dropout_2=dropout_2,
                            classifier__units_3=units_3,
                            classifier__alpha_3=alpha_3,
                            classifier__l2_kern_3=kernel_regularizer_3,
                            classifier__l2_bias_3=bias_regularizer_3,
                            classifier__dropout_3=dropout_3,
                            classifier__lr=lr,
                            classifier__epochs = epochs,
                            classifier__batch_size = batchs,
                            classifier__validation_data = validation_data,
                            classifier__callbacks=[early_stopping])
        
        # Use cross validation to monitor the deferrence between training and testing data as an evidence of overfitting
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)
        scores_train = cross_val_score(pipeline, self.X_train, self.y_train, cv=kfold, scoring="roc_auc", error_score='raise')
        mean_score_train = np.mean(scores_train)

        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)
        scores_test = cross_val_score(pipeline, self.X_test, self.y_test, cv=kfold, scoring="roc_auc", error_score='raise')
        mean_score_test = np.mean(scores_test)

        print(f"Train mean AUC score is {mean_score_train} while test mean AUC score is {mean_score_test}.")
      
        # Fit the pipeline to the training data
        history = pipeline.fit(self.X_train, self.y_train)

        #score = pipeline.evaluate(X_test, y_test, verbose=0)  

        y_pred_prob_tpe = pipeline.predict(self.X_test)

        y_pred_tpe = y_pred_prob_tpe.round()

        print(classification_report(self.y_test, y_pred_tpe, target_names = ['No','Yes']))

        acc = accuracy_score(self.y_test, y_pred_tpe)
        pre = precision_score(self.y_test, y_pred_tpe)
        rec = recall_score(self.y_test, y_pred_tpe)
        f1 = f1_score(self.y_test, y_pred_tpe)
        auc = roc_auc_score(self.y_test, y_pred_tpe)
        
        y_test_array = np.array(self.y_test)
        y_pred_tpe_array = np.array(y_pred_tpe)
        H_measure = h_score(y_test_array, y_pred_tpe_array)

        opt = [acc, pre, rec, f1, auc, H_measure]

        scores = pd.DataFrame(opt, index =['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC', "H-measure"], 
                              columns =['Tree-structured Parzen Estimator'])
        
        return scores 

## Take scores for every tuning algorithm

In [4]:
tun = tuning_methods(X_train, X_val, y_train,  y_val, X_test, y_test)
trials = 100

# Tree-structured Parzen Estimator
params_TPE = tun.Tuning(n_trials=trials, sampler = optuna.samplers.TPESampler())

[I 2023-03-02 23:04:52,824] A new study created in memory with name: no-name-3b002523-105e-45e1-b3e2-084fffb95906
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 1s 2ms/step


[I 2023-03-02 23:05:39,894] Trial 0 finished with value: 0.86895663991604 and parameters: {'units_1': 14, 'units_2': 35, 'units_3': 38, 'alpha_1': 0.08790238662958884, 'alpha_2': 0.0402434875913882, 'alpha_3': 0.044623555095021646, 'dropout_1': 0.29959674366389866, 'dropout_2': 0.46636660054341156, 'dropout_3': 0.2279736212886275, 'l2_kern_1': 0.0006624875890478167, 'l2_kern_2': 0.00020149621833749255, 'l2_kern_3': 0.005388427895754779, 'l2_bias_1': 8.346761154468321e-05, 'l2_bias_2': 0.0062271481991586644, 'l2_bias_3': 0.003218707379320239, 'lr': 0.0022220983769077086, 'batch_size': 512}. Best is trial 0 with value: 0.86895663991604.
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 23:08:26,092] Trial 1 finished with value: 0.8595117106424038 and parameters: {'units_1': 37, 'units_2': 30, 'units_3': 30, 'alpha_1': 0.03799094255598091, 'alpha_2': 0.053988497009108044, 'alpha_3': 0.041222162507640686, 'dropout_1': 0.26337383036697326, 'dropout_2': 0.24982315507362898, 'dropout_3': 0.3777525517788115, 'l2_kern_1': 0.00012542010369282663, 'l2_kern_2': 0.0023084695640879296, 'l2_kern_3': 0.00025706641530132753, 'l2_bias_1': 0.000895206977210595, 'l2_bias_2': 0.0006067071911115718, 'l2_bias_3': 3.95293380796631e-05, 'lr': 5.6006670637220445e-05, 'batch_size': 32}. Best is trial 0 with value: 0.86895663991604.
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


...

195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 00:27:30,423] Trial 98 finished with value: 0.8811478700920078 and parameters: {'units_1': 35, 'units_2': 33, 'units_3': 38, 'alpha_1': 0.0269274212012255, 'alpha_2': 0.04645011247689272, 'alpha_3': 0.0894875987469014, 'dropout_1': 0.2738205724058086, 'dropout_2': 0.22158493873374988, 'dropout_3': 0.3548387046035198, 'l2_kern_1': 0.008249097172641025, 'l2_kern_2': 0.00041652458136014084, 'l2_kern_3': 8.249304740028559e-05, 'l2_bias_1': 0.0008582044811136797, 'l2_bias_2': 0.00011803941373229845, 'l2_bias_3': 0.00014727351964057296, 'lr': 0.0004689324696797013, 'batch_size': 128}. Best is trial 71 with value: 0.887574314876599.
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 00:28:03,839] Trial 99 finished with value: 0.8764544004863756 and parameters: {'units_1': 36, 'units_2': 31, 'units_3': 35, 'alpha_1': 0.04832488946860333, 'alpha_2': 0.036944969902832234, 'alpha_3': 0.08004685267662144, 'dropout_1': 0.2436281805028918, 'dropout_2': 0.2110283705437437, 'dropout_3': 0.33169847743459824, 'l2_kern_1': 0.0037547156117273364, 'l2_kern_2': 0.0004964460113200058, 'l2_kern_3': 5.337066704416042e-05, 'l2_bias_1': 0.000496602118038529, 'l2_bias_2': 0.0002752067811999536, 'l2_bias_3': 1.1827916204025964e-05, 'lr': 0.0008592282428416903, 'batch_size': 1024}. Best is trial 71 with value: 0.887574314876599.


Number of finished trials: 100
Best trial:
  Value: 0.887574314876599
    units_1: 39
    units_2: 27
    units_3: 21
    alpha_1: 0.03448950036628276
    alpha_2: 0.06027341319928116
    alpha_3: 0.031090573312908
    dropout_1: 0.2326111837260853
    dropout_2: 0.29577418396929034
    dropout_3: 0.3417612068051909
    l2_kern_1: 0.0018313704066067636
    l2_kern_2: 2.6708068329939376e-05
    l2_kern_3: 0.0004604839899064693
    l2_bias_1: 0.0002682594007322798
    l2_bias_2: 1.7307469303378696e-05
    l2_bias_3: 0.00013645463591536674
    lr: 0.0021941934284267044
    batch_size: 128


In [6]:
tpe_scores = tun.Best_MLP_model(params_TPE)

C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:169: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 1ms/step
Train mean AUC score is 0.8880934506364998 while test mean AUC score is 0.8942457395677712.
183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          No       0.91      0.98      0.95      4648
         Yes       0.91      0.63      0.75      1186

    accuracy                           0.91      5834
   macro avg       0.91      0.81      0.85      5834
weighted avg       0.91      0.91      0.91      5834



In [7]:
# Random Sampler
trials = 100
params_rs = tun.Tuning(n_trials=trials, sampler = optuna.samplers.RandomSampler())
rs_scores = tun.Best_MLP_model(params_rs)
rs_scores = rs_scores.rename(columns={'Tree-structured Parzen Estimator': 'Random Sampler'})


[I 2023-03-03 08:42:13,640] A new study created in memory with name: no-name-89d12cc3-a2d6-458e-800a-16832e679e2a
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 08:44:54,368] Trial 0 finished with value: 0.8615781256868192 and parameters: {'units_1': 28, 'units_2': 12, 'units_3': 7, 'alpha_1': 0.015861545138969264, 'alpha_2': 0.03452138364869379, 'alpha_3': 0.01987468431905831, 'dropout_1': 0.41452899564420453, 'dropout_2': 0.2337694591027748, 'dropout_3': 0.3036323959346925, 'l2_kern_1': 0.00042146128106376345, 'l2_kern_2': 0.0005379499504354604, 'l2_kern_3': 0.008593869504619576, 'l2_bias_1': 3.241748423253626e-05, 'l2_bias_2': 0.002839174122900988, 'l2_bias_3': 2.8921944975777224e-05, 'lr': 0.0002062860359132819, 'batch_size': 16}. Best is trial 0 with value: 0.8615781256868192.
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 08:45:21,415] Trial 1 finished with value: 0.8235129044037084 and parameters: {'units_1': 10, 'units_2': 35, 'units_3': 12, 'alpha_1': 0.0581569738052718, 'alpha_2': 0.01718405241444102, 'alpha_3': 0.03303477797294086, 'dropout_1': 0.4049642279989639, 'dropout_2': 0.46691917348032463, 'dropout_3': 0.4353752366156388, 'l2_kern_1': 2.2058415285849025e-05, 'l2_kern_2': 0.0008816099272855833, 'l2_kern_3': 0.00010355823374675212, 'l2_bias_1': 8.037340346098668e-05, 'l2_bias_2': 0.004898790619087872, 'l2_bias_3': 0.0002097529005124815, 'lr': 5.535832878321009e-05, 'batch_size': 512}. Best is trial 0 with value: 0.8615781256868192.
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


...

195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 10:23:19,906] Trial 99 finished with value: 0.8431011274011674 and parameters: {'units_1': 19, 'units_2': 38, 'units_3': 2, 'alpha_1': 0.053994939524965216, 'alpha_2': 0.017125698068268753, 'alpha_3': 0.028303390384643703, 'dropout_1': 0.42939645525653414, 'dropout_2': 0.4068765927116138, 'dropout_3': 0.42593207629058205, 'l2_kern_1': 0.0020229996220260905, 'l2_kern_2': 0.00037886208071732686, 'l2_kern_3': 0.005286631080960334, 'l2_bias_1': 0.00033182598842752294, 'l2_bias_2': 0.00023188364847679605, 'l2_bias_3': 0.0015965754401939187, 'lr': 0.0007349933933612031, 'batch_size': 256}. Best is trial 44 with value: 0.8849686840471979.


Number of finished trials: 100
Best trial:
  Value: 0.8849686840471979
    units_1: 39
    units_2: 37
    units_3: 36
    alpha_1: 0.024722466594298814
    alpha_2: 0.04285142661061093
    alpha_3: 0.04115981531581119
    dropout_1: 0.23498979624736352
    dropout_2: 0.30606821307288007
    dropout_3: 0.3098524407137488
    l2_kern_1: 0.0004048176916977805
    l2_kern_2: 2.301151671773877e-05
    l2_kern_3: 0.005663263316663826
    l2_bias_1: 7.017675888772672e-05
    l2_bias_2: 0.0025186631786981812
    l2_bias_3: 0.00012329617155930765
    lr: 0.0012046149677703948
    batch_size: 256


C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:169: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8843782779513392 while test mean AUC score is 0.8840838629146512.
183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          No       0.91      0.97      0.94      4648
         Yes       0.85      0.65      0.73      1186

    accuracy                           0.90      5834
   macro avg       0.88      0.81      0.84      5834
weighted avg       0.90      0.90      0.90      5834



In [8]:
rs_scores = tun.Best_MLP_model(params_rs)
rs_scores = rs_scores.rename(columns={'Tree-structured Parzen Estimator': 'Random Sampler'})


C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:169: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 1ms/step
Train mean AUC score is 0.8841618765912067 while test mean AUC score is 0.8859078263095519.
183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          No       0.90      0.98      0.94      4648
         Yes       0.90      0.59      0.72      1186

    accuracy                           0.90      5834
   macro avg       0.90      0.79      0.83      5834
weighted avg       0.90      0.90      0.90      5834



In [9]:
# Genetic Algorithm
params_ga = tun.Tuning(n_trials=trials, sampler = optuna.samplers.NSGAIISampler())
ga_scores = tun.Best_MLP_model(params_ga)
ga_scores = ga_scores.rename(columns={'Tree-structured Parzen Estimator': 'Genetic Algorithm'})


[I 2023-03-03 10:28:36,679] A new study created in memory with name: no-name-8a02d247-6218-44d6-a9c8-40dc29540ccc
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 10:30:49,232] Trial 0 finished with value: 0.8662054127293111 and parameters: {'units_1': 22, 'units_2': 11, 'units_3': 14, 'alpha_1': 0.0768272732661319, 'alpha_2': 0.031320386561975125, 'alpha_3': 0.021596413670422157, 'dropout_1': 0.47096326798635774, 'dropout_2': 0.40017577068625376, 'dropout_3': 0.2999599844584318, 'l2_kern_1': 0.0007540243651316639, 'l2_kern_2': 1.2676803968501516e-05, 'l2_kern_3': 1.2970352764262903e-05, 'l2_bias_1': 1.225627552991124e-05, 'l2_bias_2': 4.684559016954587e-05, 'l2_bias_3': 0.00595871549597581, 'lr': 0.0005329892056112282, 'batch_size': 16}. Best is trial 0 with value: 0.8662054127293111.
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 10:31:37,378] Trial 1 finished with value: 0.8411050838632992 and parameters: {'units_1': 10, 'units_2': 3, 'units_3': 25, 'alpha_1': 0.01469681195800619, 'alpha_2': 0.04301311152882611, 'alpha_3': 0.06309375184716276, 'dropout_1': 0.3681390095416731, 'dropout_2': 0.2616548275660719, 'dropout_3': 0.34411806635932285, 'l2_kern_1': 3.394054530487117e-05, 'l2_kern_2': 0.00019371325208455365, 'l2_kern_3': 0.0004184122035401708, 'l2_bias_1': 1.8721002667293214e-05, 'l2_bias_2': 0.0004556313245183664, 'l2_bias_3': 0.00025975390762901745, 'lr': 0.00021044411343000652, 'batch_size': 32}. Best is trial 0 with value: 0.8662054127293111.
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


...

195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 12:17:16,958] Trial 99 finished with value: 0.8473282884816857 and parameters: {'units_1': 12, 'units_2': 27, 'units_3': 27, 'alpha_1': 0.03270457851799683, 'alpha_2': 0.06293759423074294, 'alpha_3': 0.051343239689101296, 'dropout_1': 0.3151373309454545, 'dropout_2': 0.4969687452999564, 'dropout_3': 0.3331119094720426, 'l2_kern_1': 0.00047196436532293554, 'l2_kern_2': 7.4332458844821e-05, 'l2_kern_3': 2.1094489158545268e-05, 'l2_bias_1': 8.424665095034688e-05, 'l2_bias_2': 0.00014301618806787715, 'l2_bias_3': 0.00017533352766397825, 'lr': 0.0007192394758225132, 'batch_size': 512}. Best is trial 28 with value: 0.8837906505609912.


Number of finished trials: 100
Best trial:
  Value: 0.8837906505609912
    units_1: 24
    units_2: 24
    units_3: 22
    alpha_1: 0.04651863386649763
    alpha_2: 0.04478061343218964
    alpha_3: 0.09255290154972129
    dropout_1: 0.2525757264352791
    dropout_2: 0.2578779070309256
    dropout_3: 0.3320117232148606
    l2_kern_1: 2.063067154873716e-05
    l2_kern_2: 4.789300964265285e-05
    l2_kern_3: 0.0025401637387441047
    l2_bias_1: 2.702725704139721e-05
    l2_bias_2: 0.0001862021998588623
    l2_bias_3: 0.0017469911824291179
    lr: 0.006756326718658271
    batch_size: 256


C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:169: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 1ms/step
Train mean AUC score is 0.8831577043780025 while test mean AUC score is 0.8892698365551803.
183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          No       0.91      0.98      0.94      4648
         Yes       0.87      0.63      0.73      1186

    accuracy                           0.91      5834
   macro avg       0.89      0.80      0.84      5834
weighted avg       0.90      0.91      0.90      5834



In [10]:
# Quasi Monte Carlo
params_mc = tun.Tuning(n_trials=trials, sampler = optuna.samplers.QMCSampler())
mc_scores = tun.Best_MLP_model(params_mc)
mc_scores = mc_scores.rename(columns={'Tree-structured Parzen Estimator': 'Quasi Monte Carlo'})


C:\Users\User\AppData\Local\Temp\ipykernel_12316\3195711531.py:2: ExperimentalWarning: QMCSampler is experimental (supported from v3.0.0). The interface can change in the future.
  params_mc = tun.Tuning(n_trials=trials, sampler = optuna.samplers.QMCSampler())
[I 2023-03-03 12:22:34,491] A new study created in memory with name: no-name-d88ee8b3-6e24-49f3-957d-60472bbeb435
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 12:22:56,320] Trial 0 finished with value: 0.8558130797614992 and parameters: {'units_1': 19, 'units_2': 5, 'units_3': 28, 'alpha_1': 0.021260913942485797, 'alpha_2': 0.03938916996410331, 'alpha_3': 0.058148775358069185, 'dropout_1': 0.4791643251249591, 'dropout_2': 0.43519634958744025, 'dropout_3': 0.3247933410516084, 'l2_kern_1': 1.430899347833043e-05, 'l2_kern_2': 0.0005845062789525349, 'l2_kern_3': 0.0001274923429228076, 'l2_bias_1': 1.3196588229003804e-05, 'l2_bias_2': 0.000329399957788416, 'l2_bias_3': 0.006183040367887882, 'lr': 0.0017444151699903493, 'batch_size': 256}. Best is trial 0 with value: 0.8558130797614992.
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))
[W 2023-03-03 12:22:5

195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 12:25:36,012] Trial 1 finished with value: 0.7087712385745055 and parameters: {'units_1': 2, 'units_2': 2, 'units_3': 2, 'alpha_1': 0.01, 'alpha_2': 0.01, 'alpha_3': 0.01, 'dropout_1': 0.2, 'dropout_2': 0.2, 'dropout_3': 0.2, 'l2_kern_1': 1.3911789913008766e-05, 'l2_kern_2': 0.0021834097622617584, 'l2_kern_3': 0.007374555140664038, 'l2_bias_1': 4.267599815121132e-05, 'l2_bias_2': 0.0013871912837915034, 'l2_bias_3': 0.0003643596385814419, 'lr': 5.000000000000004e-05, 'batch_size': 32}. Best is trial 0 with value: 0.8558130797614992.
C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))
[W 2023-03-03 12:25:36,015] The parameter 'batch_size' in trial#2 is sampled independently by using `RandomSampler` 

...

195/195 [==============================] - 0s 1ms/step


[I 2023-03-03 14:01:40,324] Trial 99 finished with value: 0.8727373025825805 and parameters: {'units_1': 32, 'units_2': 16, 'units_3': 7, 'alpha_1': 0.019140625, 'alpha_2': 0.033203125, 'alpha_3': 0.043046875000000005, 'dropout_1': 0.24453125, 'dropout_2': 0.30546875, 'dropout_3': 0.35234375, 'l2_kern_1': 0.0028902639100224507, 'l2_kern_2': 0.00010181517217181818, 'l2_kern_3': 1.4590242156305593e-05, 'l2_bias_1': 0.00011341944035027563, 'l2_bias_2': 4.782858141653789e-05, 'l2_bias_3': 0.006853895838650084, 'lr': 0.0049475985514946625, 'batch_size': 1024}. Best is trial 30 with value: 0.8818502648683421.


Number of finished trials: 100
Best trial:
  Value: 0.8818502648683421
    units_1: 32
    units_2: 32
    units_3: 27
    alpha_1: 0.0746875
    alpha_2: 0.029687500000000006
    alpha_3: 0.012812500000000001
    dropout_1: 0.22812500000000002
    dropout_2: 0.284375
    dropout_3: 0.471875
    l2_kern_1: 0.00010746078283213182
    l2_kern_2: 0.002206734069084591
    l2_kern_3: 0.0006042963902381332
    l2_bias_1: 0.0014330125702369636
    l2_bias_2: 0.0002548296747979348
    l2_bias_3: 0.00016548170999431823
    lr: 0.0011620011671182521
    batch_size: 16


C:\Users\User\AppData\Local\Temp\ipykernel_12316\574535459.py:169: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8827707667139272 while test mean AUC score is 0.8904637157166935.
183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          No       0.90      0.99      0.94      4648
         Yes       0.94      0.56      0.71      1186

    accuracy                           0.90      5834
   macro avg       0.92      0.78      0.82      5834
weighted avg       0.91      0.90      0.89      5834



## Optimal parameters comparison

In [14]:
params_all = pd.DataFrame(zip([item[1] for item in params_TPE], [item[1] for item in params_rs], 
                              [item[1] for item in params_ga], [item[1] for item in params_mc]),
                           index =['NEURONS LAYER 1 ','NEURONS LAYER 2','NEURONS LAYER 3',
                                   "AlPHA_l","AlPHA_2","AlPHA_3",
                                   'DROPOUT LAYER 1 ','DROPOUT LAYER 2', 'DROPOUT LAYER 3',
                                  "KERNEL REGULARIZER_1","KERNEL REGULARIZER_2","KERNEL REGULARIZER_3",
                                   "BIAS REGURALIZER_1","BIAS REGURALIZER_2","BIAS REGURALIZER_3",
                                  'LEARNING RATE',"BATCH SIZE", 'Time'] ,
                          columns = ['TPE','RS','GA', 'MC'])

params_all

,TPE,RS,GA,MC
NEURONS LAYER 1,39.000000,39.000000,24.000000,32.000000
NEURONS LAYER 2,27.000000,37.000000,24.000000,32.000000
NEURONS LAYER 3,21.000000,36.000000,22.000000,27.000000
AlPHA_l,0.034490,0.024722,0.046519,0.074688
AlPHA_2,0.060273,0.042851,0.044781,0.029688
AlPHA_3,0.031091,0.041160,0.092553,0.012813
DROPOUT LAYER 1,0.232611,0.234990,0.252576,0.228125
DROPOUT LAYER 2,0.295774,0.306068,0.257878,0.284375
DROPOUT LAYER 3,0.341761,0.309852,0.332012,0.471875
KERNEL REGULARIZER_1,0.001831,0.000405,0.000021,0.000107


## Metrics Summary

In [16]:
all_scores = pd.concat([tpe_scores, rs_scores, ga_scores, mc_scores], axis =1)
all_scores

,Tree-structured Parzen Estimator,Random Sampler,Genetic Algorithm,Quasi Monte Carlo
Accuracy,0.912239,0.903668,0.906754,0.904011
Precision,0.907005,0.896947,0.873256,0.938375
Recall,0.633221,0.594435,0.633221,0.564924
F1-score,0.745780,0.715010,0.734115,0.705263
AUC,0.808327,0.788504,0.804885,0.777729
H-measure,0.535375,0.491948,0.522100,0.476017


## Save results

In [17]:
# Save the clean dataset

all_scores.to_csv(r'C:\Users\...\Credit_risk_dataset_3_Layer_scores.csv', index=False)

params_all.to_csv(r'C:\Users\...\Credit_risk_dataset_3_Layer_stracture.csv', index=False)
